In [ ]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
yf.pdr_override()


tickers = si.tickers_sp500()
tickers = [item.replace(".", "-") for item in tickers] # Yahoo Finance uses dashes instead of dots
index_name = '^GSPC' # S&P 500

start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

tgt_tic = []
tgt_cr = []
tgt_de = []
tgt_roe = []
tgt_roa = []
tgt_pe = []
tgt_pb = []
annual_return = []
for tick in tickers: 
    try:
#'valq' is quarterly
        valq = si.get_balance_sheet(tick,yearly=False)
        inc = si.get_income_statement(tick,yearly=True)
        r = pdr.get_data_yahoo(tick,start_date,end_date)
        
#current ratio
        CR = si.get_stats(tick).iloc[46,1]
        debt = valq.loc['totalLiab'][0]
        equity = valq.loc['totalStockholderEquity'][0]
        de = round(debt/equity, 2)
    except:
        print('n/a')
    #return on equity; we use last year's BV
    try:
        ni = inc.loc['netIncomeApplicableToCommonShares'][0]
        total_eq = si.get_balance_sheet(tick,yearly=True).loc['totalStockholderEquity'][1]
        roe = float(round(ni/total_eq,2))
    except:
        print('n/a')
    #return on assets; last year's assets
    try:
        ta = si.get_balance_sheet('aapl',yearly=True).loc['totalAssets'][1]
        roa = float(round(ni/ta,2))
    except:
        print('n/a')
    #yearly return; we are using yahoo historical data, rounded to 2 decimals, (x1/x0)-1 format
    try:
        today = r.iloc[250,4]
        ly = r.iloc[0,4]
        ret = round(today/ly,2) -1
    except:
        print('n/a')
    #Forward P/E
    try:
        p_e = float(si.get_stats_valuation(tick).iloc[3,1])
    except:
        print('n/a')
    # Price/book value of equity; here we use current equity
    try:
        p_b = float(si.get_stats_valuation(tick).iloc[6,1])
    except:
        print('n/a')
        
    finally:
        # these are the conditions we want each stock to have in order to populate the dataframe 
        c1 = float(CR) > 1.5
        c2 = .01 < de < 1
        c3 = roe >.1
        c4 = roa > .1
        c5 = float(ret) < .3
        c6 = p_e < 15
        c7 = p_b < 2
        if c1 and c2 and c3 and c4 and c5 and c6 and c7:
            #we append each list
            tgt_tic.append(tick)
            tgt_cr.append(CR)
            tgt_de.append(de)
            tgt_roe.append(roe)
            tgt_roa.append(roa)
            tgt_pe.append(p_e)
            tgt_pb.append(p_b)
            annual_return.append(ret)
            #use each list to fill each column
        tic_dict = {"tic" : [tgt_tic], "Current Ratio" : [tgt_cr], "debt/equity" : [tgt_de], "ROE" : [tgt_roe], "ROA" : [tgt_roa], "Forward PE" : [tgt_pe], "P/B" : [tgt_pb], "Annual Return" : [annual_return]}

        frame = pd.DataFrame(data=tic_dict)
            
            